Code for Hawks Process function tranlsated from matlab from chiangwe's HawkPR  repository

## Python version

In [21]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [22]:
%pip install scipy

Note: you may need to restart the kernel to use updated packages.


*Input variables*
*(CSV Files)*

Each has 2824 counties identified by an FIPS code
1.   **Report** ;
rows: 2824 x 6 (6 locations for cases);
cols: 4 + 297 (dates) [15/02/20 - 07/12/20];
records number of cases

2. **Mobility**;
rows: 2824 (counties);
cols: 3 + 297 (dates);
records amount of mobility

3.   **Demography**;
rows: 2824;
cols: 9;
records demographic identifiers of each county




In [29]:
import numpy as np
import pandas as pd
import warnings
import time
import scipy
import random
import scipy.stats as stats
import scipy.sparse as sparse
from scipy.stats import weibull_min, poisson
from scipy.optimize import curve_fit
from scipy.sparse import csc_matrix, eye
import statsmodels.api as sm
from statsmodels.genmod.families import Poisson
from sklearn.linear_model import PoissonRegressor
from scipy import sparse
#from google.colab import drive
#drive.mount('/content/drive')

code uses a weibull distribution to model inter infection times. the parameters are updated within the code according to the expectation maximization algorithm.

In [24]:
def HawkPR(InputPath_report, InputPath_mobility, InputPath_demography, Delta, Alpha, Beta, EMitr, DaysPred, SimTimes, OutputPath_mdl, OutputPath_pred):
    warnings.filterwarnings('ignore')

    # Read in parameters
    if Alpha == '' and Beta == '':
        print('No shape and scale parameter for Weibull distribution provided. Use MLE to infer alpha and beta ... ')
        alphaScale_in = 0
        betaShape_in = 0
    else:
        alphaScale_in = float(Alpha)
        betaShape_in = float(Beta)

    if Delta == '':
        print('No shift parameter for mobility provided. It will set to zero ... ')
        mobiShift_in = 0
    else:
        mobiShift_in = int(Delta)

    # Read-in COVID data
    NYT = pd.read_csv(InputPath_report)

    # Read-in mobility
    Mobi = pd.read_csv(InputPath_mobility)

    # Read-in demographic
    Demo = pd.read_csv(InputPath_demography)
    Demo_val = Demo.iloc[:, 3:].values

    # Data pre-processing
    covid = NYT.iloc[:, 3:].values
    covid = NYT.iloc[:, 3:].apply(pd.to_numeric, errors='coerce').values  # Convert to numeric, coerce errors to NaN
    covid[np.isnan(covid)] = 0
    covid = np.hstack([np.zeros((covid.shape[0], 1)), np.diff(covid, axis=1)])
    covid[covid <= 0] = 0

    # Pad to shift
    mob_head = Mobi.iloc[:, :4]
    mob_val = Mobi.iloc[:, 4:].values

    for _ in range(mobiShift_in):
        mob_val = np.hstack([np.mean(mob_val[:, :7], axis=1, keepdims=True), mob_val])

    # Get Key and Date
    NYT_Date_list = NYT.columns[3:]
    NYT_Key_list = NYT.iloc[:, :3].values

    Mobi_Type_list = Mobi.iloc[:6, 3].values
    Mobi_Date_list = Mobi.columns[4:]
    Mobi_Key_list = Mobi.iloc[::6, :3].values

    Demo_Type_list = Demo.columns[3:]
    Demo_Key_list = Demo.iloc[:, 0].values


    n_cty, n_day = covid.shape
    n_mobitype = mob_val.shape[0] // n_cty

    print(f'There are {n_cty} counties, {n_mobitype} types of Mobility indices, and {n_day} days in the COVID reports.')

    # Train & Test Split
    n_tr = covid.shape[1]
    mob_tr = mob_val[:, :n_tr]
    mob_te = mob_val[:, n_tr:n_tr+DaysPred]

    # Normalization
    mob_tr_reshape = mob_tr.reshape(n_mobitype,-1).T
    mob_te_reshape = mob_te.reshape(n_mobitype, -1).T

    Demo_val_in = Demo_val
    Demo_val_tr = np.tile(Demo_val_in, (n_tr, 1))
    Demo_val_te = np.tile(Demo_val_in, (DaysPred, 1))

    covid_tr = covid

    Covar_tr = np.hstack([mob_tr_reshape, Demo_val_tr])
    Covar_te = np.hstack([mob_te_reshape, Demo_val_te])

    Covar_tr_mean = np.mean(Covar_tr, axis=0)
    Covar_tr_std = np.std(Covar_tr, axis=0)

    Covar_tr = (Covar_tr - Covar_tr_mean) / Covar_tr_std
    Covar_te = (Covar_te - Covar_tr_mean) / Covar_tr_std

    # Get Variable names
    #clean up variable names
    VarNamesOld = np.concatenate([Mobi_Type_list, Demo_Type_list.T, ['Qprob']])
    VarNames = [name.replace(' & ', '_').replace(' ', '_').lstrip('_') for name in VarNamesOld]

    # Define Parameters
    n_day_tr = n_day
    T = n_day_tr
    dry_correct = 2

    emiter = EMitr
    break_diff = 1e-3
    day_for_tr = min(T - dry_correct, mob_tr.shape[1])

    # Initialize Inferred Parameters
    if (alphaScale_in == 0) and (betaShape_in == 0):
        alpha = 2
        beta = 2
    else:
        alpha = alphaScale_in
        beta = betaShape_in

    # Initial Weibull values
    wbl_val = np.tile(np.tril(weibull_min.pdf(np.arange(1, n_day_tr+1)[:,None] - np.arange(1, n_day_tr+1), c=beta, loc=0, scale=alpha)), (n_cty, 1))

    # K0 reproduction number, a function of time and mobility.
    # K0 is a n_county * n_day by n_day matrix.
    K0 = np.ones((n_cty, n_day_tr))
    K0_ext_j = np.repeat(K0, n_day_tr, axis=0)

    # q is a n_county * n_day by n_day matrix.
    q = sparse.lil_matrix((n_cty * n_day_tr, n_day_tr))

    # Mu is the background rate
    mus = 0.5 * np.ones(n_cty)
    mus = mus.reshape(n_cty , 1)

    # lam is the event intensity
    lam = np.zeros((n_cty, T))

    '''
    Debug lines
    print(f'shape of wbl_val: {wbl_val.shape})')
    print(f'shape of K0(reproductive number): {K0.shape})')
    print(f'shape of K0_ext_j(adjusted): {K0_ext_j.shape})')
    print(f'mus shape is: {mus.shape}')

    '''

    # EM iteration
    alpha_delta = []
    alpha_prev = []
    beta_delta = []
    beta_prev = []
    mus_delta = []
    mus_prev = []
    K0_delta = []
    K0_prev = []
    theta_delta = []
    theta_prev = []

    for itr in range(emiter):
        start_time = time.time()

        # E-step
        '''
        Expectation step
        - given the parameters 0, alpha, beta and mus_c estimated from last iteration
        - estimate latent variables p_c(i,j) for each county
        - calculated using formula:

        p_c(i,j) = R(x_c^t_j-delta, 0) w(t_i - t_j |alpha,beta) / lam_c(t_i)
        {probability that injection i causes j}

        p_c(i,i) = mus_c/ lam_c(t_i)
        {probability that infection is imported}

        '''
        q = K0_ext_j * wbl_val * (covid_tr_ext_j(covid_tr, n_day_tr) > 0)

        eye_mu = np.tile(np.identity(n_day_tr), (n_cty,1)) * np.tile(mus, (n_day_tr,1))
        #eye_mu = np.kron(sparse.eye(n_day_tr), np.ones((n_cty, 1))) * np.tile(mus,(n_day_tr,1))
        #eye_mu = eye_mu.toarray()
        lam = np.sum(q * covid_tr_ext_j(covid_tr, n_day_tr) + eye_mu, axis=1)
        lam = lam.reshape(lam.shape[0],1)
        lam_eq_zero = lam == 0

        q = np.divide(q,lam)
        q[lam_eq_zero.flatten()] = 0

        lam = lam.reshape(n_day_tr, n_cty).T
        '''
        print(f'shape of q: {q.shape}')
        print(f'shape of covid_tr: {covid_tr_ext_j(covid_tr, n_day_tr).shape}')
        print(f'shape of eye_mu: {eye_mu.shape}')
        print(f'shape of lam: {lam.shape}')

        '''

        # M-step
        '''
        Maximisation step

        maximise the data log likelihood wrt model parameters, done in three different opimisation problems
        1) coefficient 0 from poisson regression
        2) maximum likelihood estimation for shape and scale parameters (alpha and beta)
        3) background rate mu determined analytically
        '''
        # Calculate Q, which stands for the average number (observed) of children generated by a SINGLE event j
        # Note that the last "dry_correct" days of Q will be accurate, since we haven't observed their children yet
        Q = np.reshape(q * covid_tr_ext_i(covid_tr, n_day_tr, n_cty), (n_day_tr, n_day_tr * n_cty))
        Q = np.reshape(np.sum(Q, axis=0), (n_cty, n_day_tr))

        # M-step Part 2)
        # Weibull fitting
        if (alphaScale_in == 0) and (betaShape_in == 0):
            # Create the observation matrix
            obs = np.tril(np.arange(1, day_for_tr + 1).reshape(-1, 1) - np.arange(1, day_for_tr + 1), -1).toarray()

            # Calculate the frequency
            freq = covid_tr_ext_j(covid_tr, n_day_tr) * covid_tr_ext_i(covid_tr, n_day_tr, n_cty) * q
            freq = freq.toarray() if isinstance(freq, sparse.csr_matrix) else freq
            freq = np.sum(np.transpose(freq.reshape(n_day_tr, n_cty, n_day_tr), (0, 2, 1)), axis=2)
            freq = freq[:day_for_tr, :day_for_tr]

            # Find indices where both obs and freq are positive
            Ind_ret = np.where((obs > 0) & (freq > 0))
            obs = obs[Ind_ret]
            freq = freq[Ind_ret]

            # Fit Weibull
            def neg_log_likelihood(params):
                alpha, beta = params
                return -np.sum(freq * weibull_min.logpdf(obs, beta, scale=alpha))

            # Initial guess for alpha and beta
            initial_guess = [1.0, 1.0]

            # Minimize the negative log likelihood
            result = scipy.optimize.minimize(neg_log_likelihood, initial_guess, method='L-BFGS-B')
            alpha, beta = result.x

            # Create wbl_val
            wbl_val = np.tile(np.tril(weibull_min.pdf(np.arange(1, n_day_tr + 1)[:,None] - np.arange(1, n_day_tr + 1), c=beta, scale=alpha)),(n_cty, 1))
            #print(f'shape of wbl_val after fitting: {wbl_val.shape}')

        # Estimate K0 and Coefficients for Poisson regression
        glm_tr = np.nan_to_num(Covar_tr[:n_cty*day_for_tr, :])
        glm_y = np.nan_to_num(Q[:, :day_for_tr].reshape(-1,1))
        print(f'shape of glm_tr: {glm_tr.shape}')

        glm_tr = sm.add_constant(glm_tr)

        freqs = covid_tr[: , :day_for_tr].reshape(-1,1)
        freqs = freqs.flatten()
        np.nan_to_num(freqs)
        print(f'shape of freqs: {freqs.shape}')

        family = Poisson()
        # Define the GLM model with cleaned data and specify missing='drop'
        model = sm.GLM(glm_y, glm_tr, family=family, freq_weights=freqs, missing='drop')
        print(model)

        # Fit the model
        result = model.fit(maxiter=300)
        #print(result.summary())
        '''
        print(f' shape of result.params i.e. model coefficients = {(result.params).shape}')
        print(f'shape of Covar_tr: {Covar_tr.shape}')
        print(f'day_for_tr: {day_for_tr}')

        '''

        # M-step Part1)

        ypred = result.predict(sm.add_constant(Covar_tr))

        # Reshape ypred to match the original dimensions
        K0 = np.reshape(ypred, (n_cty, n_day_tr))

        # Bound K0
        K0 = scipy.signal.savgol_filter(K0, window_length=5, polyorder=2)  # Adjust window_length and polyorder as needed
        print(f'shape of bound K0: {K0.shape}')
        K0_ext_j = np.repeat(K0, n_day_tr, axis=0)

        # M-step Part 3)
        # Estimate mu, the background rate

        # Find where lam is zero
        lam_eq_zero = np.where(lam[:, :day_for_tr] == 0)
        # Estimate mu by dividing mus by lam
        mus = mus / lam[:, :day_for_tr]

        # Set mus to 0 where lam is 0
        mus[lam_eq_zero] = 0
        # Calculate the average of mus weighted by covid_tr
        mus = np.sum(mus * covid_tr[:, :day_for_tr], axis=1) / day_for_tr
        mus = mus.reshape(mus.shape[0],1)
        #print(f'shape of mus (after M): {mus.shape}')
        #print(mus)

        # Convergence check
        if itr == 0:
            #save the first value
            alpha_prev = alpha
            beta_prev = beta
            mus_prev = mus
            K0_prev = K0
            theta_prev = np.array(result.params)
        else:
            #calculate the RMSR
            alpha_delta = np.hstack((alpha_delta, np.sqrt((alpha - alpha_prev)**2)))
            beta_delta = np.hstack((beta_delta, np.sqrt((beta - beta_prev)**2)))
            mus_delta = np.hstack((mus_delta, np.sqrt(np.mean((mus_prev - mus)**2))))
            K0_delta = np.hstack((K0_delta, np.sqrt(np.mean((K0_prev - K0)**2))))
            theta_delta = np.hstack((theta_delta, np.sqrt((np.sum((theta_prev - np.array(result.params))**2)) / len(np.array(result.params)))))

            #save the current
            alpha_prev = alpha
            beta_prev = beta
            mus_prev = mus
            K0_prev = K0
            theta_prev = np.array(result.params)
            

        # Early Stop
        if itr > 5:
          # Check the last 5 elements
          rule = (np.all(alpha_delta[-5:] < break_diff) and 
                  np.all(beta_delta[-5:] < break_diff) and 
                  np.all(mus_delta[-5:] < break_diff) and 
                  np.all(K0_delta[-5:] < break_diff) and 
                  np.all(theta_delta[-5:] < break_diff))
          
          if rule:
              print("Convergence Criterion Met. Breaking out of EM iteration...")
              break

        elapsed_time = time.time() - start_time
        print(f"ITR: Iteration {itr+1}, Elapsed time: {elapsed_time:.2f} seconds ------------------------------------------------")

    if itr == emiter - 1:
        print('Reached maximum EM iteration.')


    print(f"---- E M Algorithm Completed -----------------------------------------------------------------------------------")
    # Start Simulation
    np.savez(OutputPath_mdl, mus=mus, alpha=alpha, beta=beta, K0=K0, VarNames=VarNames, alpha_delta=alpha_delta, beta_delta=beta_delta, mus_delta=mus_delta, K0_delta=K0_delta, theta_delta=theta_delta)
    loaded_data = np.load(OutputPath_mdl + '.npz')

    # Get K0
    print(f'shape of Covar_tr: {Covar_tr.shape}')
    print(f'shape of Covar_te: {Covar_te.shape}')
    Covar_all = np.vstack((Covar_tr, Covar_te))
    n_day = n_day_tr + DaysPred
    T_sim = n_day
    Tlow = T_sim - DaysPred

    # Predict
    print(model.exog_names)
    ypred = result.predict(sm.add_constant(Covar_all))
    fK0 = ypred.reshape(n_cty, n_day)

    # Make fK0 stable
    fK0[fK0 > 4] = 4

    # Simulation results
    sim = np.zeros((n_cty, T_sim, SimTimes))

    # Simulate offsprings
    n_per_batch = 10**2
    K0_sim = fK0[:, Tlow:]

    for itr in range(SimTimes):
        np.random.seed(itr)

        # Calculate base rate
        base = np.zeros((n_cty, DaysPred))
        n_exh = np.zeros((n_cty, DaysPred))

        t_stamps = np.arange(Tlow + 1, T_sim + 1)[:, None] - np.arange(1, Tlow + 1)  
        intense = (np.tile(weibull_min.pdf(t_stamps, beta, scale=alpha), (n_cty, 1, 1)) * np.tile(fK0[:, :Tlow].reshape(n_cty, 1, Tlow), (1, DaysPred, 1)) *
        np.tile(covid_tr[:, :Tlow].reshape(n_cty, 1, Tlow), (1, DaysPred, 1)))
        
        base = np.sum(intense, axis=2) + mus
        n_exh = np.random.poisson(base)

        for itr_cty in range(int(np.ceil(n_cty * 0.5))):
            for itr_d in range(DaysPred):
                max_d = DaysPred - itr_d

                # Sample first
                if n_exh[itr_cty, itr_d] > n_per_batch:
                    n_batch = n_exh[itr_cty, itr_d] // n_per_batch
                    cand = np.random.poisson(K0_sim[itr_cty, itr_d], size=n_per_batch)
                    n_mod = n_exh[itr_cty, itr_d] % n_per_batch
                    n_offs = np.sum(cand) * n_batch + np.sum(np.random.poisson(K0_sim[itr_cty, itr_d], size=n_mod))
                else:
                    n_offs = np.sum(np.random.poisson(K0_sim[itr_cty, itr_d], size=n_exh[itr_cty, itr_d]))

                if n_offs > n_per_batch:
                    n_batch = n_offs // n_per_batch
                    n_mod = n_offs % n_per_batch

                    sim_cand_wbl = np.ceil(weibull_min.rvs(alpha, scale=beta, size=n_per_batch))
                    sim_cand_wbl = sim_cand_wbl[sim_cand_wbl <= max_d]
                    sim_cand_wbl = np.histogram(sim_cand_wbl, bins=np.arange(1, max_d + 2))[0]

                    t_delta = np.ceil(weibull_min.rvs(alpha, scale=beta, size=n_mod))
                    t_delta = t_delta[t_delta <= max_d]
                    nt = np.histogram(t_delta, bins=np.arange(1, max_d + 2))[0] + sim_cand_wbl * n_batch
                else:
                    t_delta = np.ceil(weibull_min.rvs(alpha, scale=beta, size=n_offs))
                    t_delta = t_delta[t_delta <= max_d]
                    nt = np.histogram(t_delta, bins=np.arange(1, max_d + 2))[0]

                '''
                Debugging
                print(f'Outer loop: itr_cty : {itr_cty}')
                print(f'Ineer loop: itr_day: {itr_d}')
                print(f'shape of n_exh (which is number of offspring events: {n_exh.shape})')
                print(f'First if-else block: {(n_exh[itr_cty, itr_d] > n_per_batch)}')
                print(f'Second if-else block: {n_offs > n_per_batch}')
                print(f'shape of nt: {nt.shape}')
                n_exh[itr_cty, itr_d:] += nt
                '''

        sim[:, :, itr] = np.concatenate((covid_tr, n_exh), axis=1)

    sim_out = sim[:, -DaysPred:, :]

    # Format the output
    sim_mean = np.mean(sim_out, axis=2)
    print (sim_mean)
    #Date_pred = pd.date_range(start=NYT_Date_list[-1], periods=DaysPred, freq='D').strftime('%Y_%m_%d').to_list()
    #table_out = pd.DataFrame(sim_mean, columns=Date_pred)
    #table_out = pd.concat([NYT.iloc[:, :3], table_out], axis=1)

    #table_out.to_csv(OutputPath_pred, index=False)

def covid_tr_ext_j(covid_tr, n_day_tr):
    return np.repeat(covid_tr, n_day_tr, axis=0)

def covid_tr_ext_i(covid_tr, n_day_tr, n_cty):
    return np.tile(covid_tr.T, (1, n_day_tr)).T

In [25]:
# Path to your CSV file in Google Drive
InputPath_demography = r"C:\Users\dipiy\OneDrive\Documents\GitHub\CausalSTPP\TestData\Demography_Test.csv"
InputPath_report = r"C:\Users\dipiy\OneDrive\Documents\GitHub\CausalSTPP\TestData\Report_Test2.csv"
InputPath_mobility = r"C:\Users\dipiy\OneDrive\Documents\GitHub\CausalSTPP\TestData\Mobility_Test.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(InputPath_mobility)

print(df.head())
num_rows, num_columns = df.shape
print(f'Number of rows: {num_rows}')
print(f'Number of columns: {num_columns}')

   FIPS       State              County                Type  x2020-02-15  \
0  6037  California  Los_Angeles_County   _Grocery_pharmacy            0   
1  6037  California  Los_Angeles_County              _Parks           13   
2  6037  California  Los_Angeles_County        _Residential            0   
3  6037  California  Los_Angeles_County  _Retail_recreation            1   
4  6037  California  Los_Angeles_County   _Transit_stations           -1   

   x2020-02-16  x2020-02-17  x2020-02-18  x2020-02-19  x2020-02-20  ...  \
0           -1            0            0           -1            0  ...   
1           27           30           11           13           11  ...   
2           -1            7            0            0           -1  ...   
3            4            7           -1           -1            1  ...   
4           -1          -11            2            1            2  ...   

   x2020-02-22  x2020-02-23  x2020-02-24  x2020-02-24.1  x2020-02-25  \
0           -1      

In [26]:
#scale of weibull
Alpha = 4
#shape of weibull
Beta = 2

# num of maximum iterations for EM algortihm in case convergence not reached
EMitr = 20

#additional days to be predicted by trained hawks process model
DaysPred = 6

#mobility shift parameter: ???
Delta = 3

SimTimes = 6

#to_csv function will automatically create a csv file with this path
OutputPath_pred = '/content/drive/My Drive/HawkPR_data_sim/Output.csv'
OutputPath_mdl = r"C:\Users\dipiy\OneDrive\Documents\GitHub\CausalSTPP\TestData"

In [30]:
HawkPR(InputPath_report, InputPath_mobility, InputPath_demography, Delta, Alpha, Beta, EMitr, DaysPred, SimTimes, OutputPath_mdl, OutputPath_pred)

There are 20 counties, 6 types of Mobility indices, and 10 days in the COVID reports.
shape of glm_tr: (160, 12)
shape of freqs: (160,)
shape of bound K0: (20, 10)
ITR: Iteration 1, Elapsed time: 0.07 seconds ------------------------------------------------
shape of glm_tr: (160, 12)
shape of freqs: (160,)
shape of bound K0: (20, 10)
ITR: Iteration 2, Elapsed time: 0.02 seconds ------------------------------------------------
shape of glm_tr: (160, 12)
shape of freqs: (160,)
shape of bound K0: (20, 10)
ITR: Iteration 3, Elapsed time: 0.02 seconds ------------------------------------------------
shape of glm_tr: (160, 12)
shape of freqs: (160,)
shape of bound K0: (20, 10)
ITR: Iteration 4, Elapsed time: 0.01 seconds ------------------------------------------------
shape of glm_tr: (160, 12)
shape of freqs: (160,)
shape of bound K0: (20, 10)
ITR: Iteration 5, Elapsed time: 0.01 seconds ------------------------------------------------
shape of glm_tr: (160, 12)
shape of freqs: (160,)
shap